In [33]:
import numpy as np
import pandas as pd

import datetime
import time

import random

# For plotting
%matplotlib inline

import matplotlib.pyplot as plt
import seaborn as sns

sns.set_style('darkgrid', {'axes.facecolor': '0.9'})


In [34]:
transactions_df = pd.read_csv('simulated-data-raw.csv')

In [35]:
transactions_df.head()

,Unnamed: 0,TRANSACTION_ID,TX_DATETIME,CUSTOMER_ID,TERMINAL_ID,TX_AMOUNT,TX_TIME_SECONDS,TX_TIME_DAYS,TX_FRAUD
0,0,0,2022-01-01 00:00:17,6160,6202,31.83,17,0,0
1,1,1,2022-01-01 00:00:31,596,6532,57.16,31,0,0
2,2,2,2022-01-01 00:01:05,9339,10651,28.92,65,0,0
3,3,3,2022-01-01 00:02:10,4961,8673,81.51,130,0,0
4,4,4,2022-01-01 00:02:21,6170,4884,25.17,141,0,0


In [36]:
transactions_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2928154 entries, 0 to 2928153
Data columns (total 9 columns):
 #   Column           Dtype  
---  ------           -----  
 0   Unnamed: 0       int64  
 1   TRANSACTION_ID   int64  
 2   TX_DATETIME      object 
 3   CUSTOMER_ID      int64  
 4   TERMINAL_ID      int64  
 5   TX_AMOUNT        float64
 6   TX_TIME_SECONDS  int64  
 7   TX_TIME_DAYS     int64  
 8   TX_FRAUD         int64  
dtypes: float64(1), int64(7), object(1)
memory usage: 201.1+ MB


## Date and time transformations

Two two new binary features will be created from the transaction dates and times:

* The first will characterize whether a transaction occurs during a weekday (value 0) or a weekend (1), and will be called `TX_DURING_WEEKEND`
* The second will characterize whether a transaction occurs during the day or during the day (0) or during the night (1). The night is defined as hours that are between 0pm and 6am. It will be called `TX_DURING_NIGHT`. 

For the `TX_DURING_WEEKEND` feature, a function was defined called `is_weekend`, it takes as input a Panda timestamp, and returns 1 if the date is during a weekend, or 0 otherwise. The timestamp object conveniently provides the `weekday` function to help in computing this value.

In [37]:
transactions_df.TX_DATETIME = pd.to_datetime(transactions_df["TX_DATETIME"])

In [38]:
transactions_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2928154 entries, 0 to 2928153
Data columns (total 9 columns):
 #   Column           Dtype         
---  ------           -----         
 0   Unnamed: 0       int64         
 1   TRANSACTION_ID   int64         
 2   TX_DATETIME      datetime64[ns]
 3   CUSTOMER_ID      int64         
 4   TERMINAL_ID      int64         
 5   TX_AMOUNT        float64       
 6   TX_TIME_SECONDS  int64         
 7   TX_TIME_DAYS     int64         
 8   TX_FRAUD         int64         
dtypes: datetime64[ns](1), float64(1), int64(7)
memory usage: 201.1 MB


In [39]:
def is_weekend(tx_datetime):
    
    # Transform date into weekday (0 is Monday, 6 is Sunday)
    weekday = tx_datetime.weekday()
    # Binary value: 0 if weekday, 1 if weekend
    is_weekend = weekday>=5
    
    return int(is_weekend)


In [40]:
%time transactions_df['TX_DURING_WEEKEND']=transactions_df.TX_DATETIME.apply(is_weekend)

Wall time: 5.7 s


The same principle was followed to implement the `TX_DURING_NIGHT` feature. First, a function `is_night` that takes as input a Panda timestamp, and returns 1 if the time is during the night, or 0 otherwise. The timestamp object conveniently provides the hour property to help in computing this value.

In [41]:
def is_night(tx_datetime):
    
    # Get the hour of the transaction
    tx_hour = tx_datetime.hour
    # Binary value: 1 if hour less than 6, and 0 otherwise
    is_night = tx_hour<=6
    
    return int(is_night)

In [42]:
%time transactions_df['TX_DURING_NIGHT']=transactions_df.TX_DATETIME.apply(is_night)

Wall time: 5.33 s


In [43]:
transactions_df[transactions_df.TX_TIME_DAYS>=30]

,Unnamed: 0,TRANSACTION_ID,TX_DATETIME,CUSTOMER_ID,TERMINAL_ID,TX_AMOUNT,TX_TIME_SECONDS,TX_TIME_DAYS,TX_FRAUD,TX_DURING_WEEKEND,TX_DURING_NIGHT
582499,582499,582499,2022-01-31 00:01:21,3546,5354,18.71,2592081,30,0,0,1
582500,582500,582500,2022-01-31 00:01:36,9553,8924,13.26,2592096,30,0,0,1
582501,582501,582501,2022-01-31 00:01:48,206,8647,18.60,2592108,30,0,0,1
582502,582502,582502,2022-01-31 00:01:50,7481,13441,94.75,2592110,30,0,0,1
582503,582503,582503,2022-01-31 00:01:58,9859,19773,32.07,2592118,30,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...
2928149,2928149,2928149,2022-05-31 23:59:38,8127,3133,39.82,13046378,150,0,0,0
2928150,2928150,2928150,2022-05-31 23:59:43,9916,1138,103.88,13046383,150,0,0,0
2928151,2928151,2928151,2022-05-31 23:59:46,2538,11932,47.92,13046386,150,0,0,0
2928152,2928152,2928152,2022-05-31 23:59:50,8113,18820,117.70,13046390,150,0,0,0


## Customer ID transformations

This will be done using the RFM (Recency, Frequency, Monetary value) framework proposed in {cite}`(Van Vlasselaer et al., 2015)`, to compute two of these features over three time windows. The first feature will be the number of transactions that occur within a time window (Frequency). 

The second will be the average amount spent in these transactions (Monetary value). The time windows will be set to one, seven, and thirty days. This will generate six new features. 

This will be implemented by writing a `get_customer_spending_behaviour_features` function. The function takes as inputs the set of transactions for a customer and a set of window sizes. It returns a DataFrame with the six new features. Our implementation relies on the Panda `rolling` function, which makes easy the computation of aggregates over a time window.


In [44]:
def get_customer_spending_behaviour_features(customer_transactions, windows_size_in_days=[1,7,30]):
    
    # Let us first order transactions chronologically
    customer_transactions=customer_transactions.sort_values('TX_DATETIME')
    
    # The transaction date and time is set as the index, which will allow the use of the rolling function 
    customer_transactions.index=customer_transactions.TX_DATETIME
    
    # For each window size
    for window_size in windows_size_in_days:
        
        # Compute the sum of the transaction amounts and the number of transactions for the given window size
        SUM_AMOUNT_TX_WINDOW=customer_transactions['TX_AMOUNT'].rolling(str(window_size)+'d').sum()
        NB_TX_WINDOW=customer_transactions['TX_AMOUNT'].rolling(str(window_size)+'d').count()
    
        # Compute the average transaction amount for the given window size
        # NB_TX_WINDOW is always >0 since current transaction is always included
        AVG_AMOUNT_TX_WINDOW=SUM_AMOUNT_TX_WINDOW/NB_TX_WINDOW
    
        # Save feature values
        customer_transactions['CUSTOMER_ID_NB_TX_'+str(window_size)+'DAY_WINDOW']=list(NB_TX_WINDOW)
        customer_transactions['CUSTOMER_ID_AVG_AMOUNT_'+str(window_size)+'DAY_WINDOW']=list(AVG_AMOUNT_TX_WINDOW)
    
    # Reindex according to transaction IDs
    customer_transactions.index=customer_transactions.TRANSACTION_ID
        
    # And return the dataframe with the new features
    return customer_transactions


In [45]:
%time transactions_df=transactions_df.groupby('CUSTOMER_ID').apply(lambda x: get_customer_spending_behaviour_features(x, windows_size_in_days=[1,7,30]))
transactions_df=transactions_df.sort_values('TX_DATETIME').reset_index(drop=True)

Wall time: 59.6 s


In [51]:
transactions_df.head()

,Unnamed: 0,TRANSACTION_ID,TX_DATETIME,CUSTOMER_ID,TERMINAL_ID,TX_AMOUNT,TX_TIME_SECONDS,TX_TIME_DAYS,TX_FRAUD,TX_DURING_WEEKEND,...,CUSTOMER_ID_NB_TX_7DAY_WINDOW,CUSTOMER_ID_AVG_AMOUNT_7DAY_WINDOW,CUSTOMER_ID_NB_TX_30DAY_WINDOW,CUSTOMER_ID_AVG_AMOUNT_30DAY_WINDOW,TERMINAL_ID_NB_TX_1DAY_WINDOW,TERMINAL_ID_RISK_1DAY_WINDOW,TERMINAL_ID_NB_TX_7DAY_WINDOW,TERMINAL_ID_RISK_7DAY_WINDOW,TERMINAL_ID_NB_TX_30DAY_WINDOW,TERMINAL_ID_RISK_30DAY_WINDOW
0,0,0,2022-01-01 00:00:17,6160,6202,31.83,17,0,0,1,...,1.0,31.83,1.0,31.83,0.0,0.0,0.0,0.0,0.0,0.0
1,1,1,2022-01-01 00:00:31,596,6532,57.16,31,0,0,1,...,1.0,57.16,1.0,57.16,0.0,0.0,0.0,0.0,0.0,0.0
2,2,2,2022-01-01 00:01:05,9339,10651,28.92,65,0,0,1,...,1.0,28.92,1.0,28.92,0.0,0.0,0.0,0.0,0.0,0.0
3,3,3,2022-01-01 00:02:10,4961,8673,81.51,130,0,0,1,...,1.0,81.51,1.0,81.51,0.0,0.0,0.0,0.0,0.0,0.0
4,4,4,2022-01-01 00:02:21,6170,4884,25.17,141,0,0,1,...,1.0,25.17,1.0,25.17,0.0,0.0,0.0,0.0,0.0,0.0


## Terminal ID transformations

The main goal will be to extract a *risk score*, that assesses the exposure of a given terminal ID to fraudulent transactions. The risk score will be defined as the average number of fraudulent transactions that occurred on a terminal ID over a time window. As for customer ID transformations, we will use three window sizes, of 1, 7, and 30 days.

Contrary to customer ID transformations, the time windows will not directly precede a given transaction. Instead, they will be shifted back by a *delay period*. The delay period accounts for the fact that, in practice, the fraudulent transactions are only discovered after a fraud investigation or a customer complaint. Hence, the fraudulent labels, which are needed to compute the risk score, are only available after this delay period. To a first approximation, this delay period will be set to one week. 

To perform the computation of the risk scores, a function `get_count_risk_rolling_window` was defined. The function takes as inputs the DataFrame of transactions for a given terminal ID, the delay period, and a list of window sizes. In the first stage, the number of transactions and fraudulent transactions are computed for the delay period (`NB_TX_DELAY` and `NB_FRAUD_DELAY`). In the second stage, the number of transactions and fraudulent transactions are computed for each window size plus the delay period (`NB_TX_DELAY_WINDOW` and `NB_FRAUD_DELAY_WINDOW`). The number of transactions and fraudulent transactions that occurred for a given window size, shifted back by the delay period, is then obtained by simply computing the differences of the quantities obtained for the delay period, and the window size plus delay period:

```
NB_FRAUD_WINDOW=NB_FRAUD_DELAY_WINDOW-NB_FRAUD_DELAY
NB_TX_WINDOW=NB_TX_DELAY_WINDOW-NB_TX_DELAY
```

The risk score is finally obtained by computing the proportion of fraudulent transactions for each window size (or 0 if no transaction occurred for the given window):

```
RISK_WINDOW=NB_FRAUD_WINDOW/NB_TX_WINDOW
```

Additionally to the risk score, the function also returns the number of transactions for each window size. This results in the addition of six new features: The risk and number of transactions, for three window sizes.


In [47]:
def get_count_risk_rolling_window(terminal_transactions, delay_period=7, windows_size_in_days=[1,7,30], feature="TERMINAL_ID"):
    
    terminal_transactions=terminal_transactions.sort_values('TX_DATETIME')
    
    terminal_transactions.index=terminal_transactions.TX_DATETIME
    
    NB_FRAUD_DELAY=terminal_transactions['TX_FRAUD'].rolling(str(delay_period)+'d').sum()
    NB_TX_DELAY=terminal_transactions['TX_FRAUD'].rolling(str(delay_period)+'d').count()
    
    for window_size in windows_size_in_days:
    
        NB_FRAUD_DELAY_WINDOW=terminal_transactions['TX_FRAUD'].rolling(str(delay_period+window_size)+'d').sum()
        NB_TX_DELAY_WINDOW=terminal_transactions['TX_FRAUD'].rolling(str(delay_period+window_size)+'d').count()
    
        NB_FRAUD_WINDOW=NB_FRAUD_DELAY_WINDOW-NB_FRAUD_DELAY
        NB_TX_WINDOW=NB_TX_DELAY_WINDOW-NB_TX_DELAY
    
        RISK_WINDOW=NB_FRAUD_WINDOW/NB_TX_WINDOW
        
        terminal_transactions[feature+'_NB_TX_'+str(window_size)+'DAY_WINDOW']=list(NB_TX_WINDOW)
        terminal_transactions[feature+'_RISK_'+str(window_size)+'DAY_WINDOW']=list(RISK_WINDOW)
        
    terminal_transactions.index=terminal_transactions.TRANSACTION_ID
    
    # Replace NA values with 0 (all undefined risk scores where NB_TX_WINDOW is 0) 
    terminal_transactions.fillna(0,inplace=True)
    
    return terminal_transactions


In [48]:
transactions_df[transactions_df.TX_FRAUD==1]

,Unnamed: 0,TRANSACTION_ID,TX_DATETIME,CUSTOMER_ID,TERMINAL_ID,TX_AMOUNT,TX_TIME_SECONDS,TX_TIME_DAYS,TX_FRAUD,TX_DURING_WEEKEND,TX_DURING_NIGHT,CUSTOMER_ID_NB_TX_1DAY_WINDOW,CUSTOMER_ID_AVG_AMOUNT_1DAY_WINDOW,CUSTOMER_ID_NB_TX_7DAY_WINDOW,CUSTOMER_ID_AVG_AMOUNT_7DAY_WINDOW,CUSTOMER_ID_NB_TX_30DAY_WINDOW,CUSTOMER_ID_AVG_AMOUNT_30DAY_WINDOW
1320,1320,1320,2022-01-01 04:19:14,7940,3321,225.94,15554,0,1,1,1,1.0,225.940000,1.0,225.940000,1.0,225.940000
7223,7223,7223,2022-01-01 10:17:43,3774,5265,225.41,37063,0,1,1,0,3.0,158.073333,3.0,158.073333,3.0,158.073333
11880,11879,11879,2022-01-01 13:31:48,4944,12875,222.26,48708,0,1,1,0,2.0,127.605000,2.0,127.605000,2.0,127.605000
12816,12816,12816,2022-01-01 14:13:42,5717,19134,27.65,51222,0,1,1,0,4.0,45.180000,4.0,45.180000,4.0,45.180000
13192,13192,13192,2022-01-01 14:30:41,9394,16316,86.90,52241,0,1,1,0,2.0,52.145000,2.0,52.145000,2.0,52.145000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2927351,2927351,2927351,2022-05-31 20:53:43,8981,11559,64.44,13035223,150,1,0,0,1.0,64.440000,1.0,64.440000,8.0,79.667500
2927484,2927484,2927484,2022-05-31 21:13:26,8412,3424,216.15,13036406,150,1,0,0,2.0,188.700000,18.0,53.166111,83.0,33.459157
2927672,2927672,2927672,2022-05-31 21:47:38,4933,9885,78.63,13038458,150,1,0,0,5.0,95.184000,30.0,84.165333,112.0,83.000982
2927774,2927774,2927774,2022-05-31 22:06:43,9615,7377,737.15,13039603,150,1,0,0,5.0,367.432000,27.0,287.218519,117.0,154.874957


In [49]:
%time transactions_df=transactions_df.groupby('TERMINAL_ID').apply(lambda x: get_count_risk_rolling_window(x, delay_period=7, windows_size_in_days=[1,7,30], feature="TERMINAL_ID"))
transactions_df=transactions_df.sort_values('TX_DATETIME').reset_index(drop=True)


Wall time: 4min 54s


In [50]:
transactions_df.head()

,Unnamed: 0,TRANSACTION_ID,TX_DATETIME,CUSTOMER_ID,TERMINAL_ID,TX_AMOUNT,TX_TIME_SECONDS,TX_TIME_DAYS,TX_FRAUD,TX_DURING_WEEKEND,...,CUSTOMER_ID_NB_TX_7DAY_WINDOW,CUSTOMER_ID_AVG_AMOUNT_7DAY_WINDOW,CUSTOMER_ID_NB_TX_30DAY_WINDOW,CUSTOMER_ID_AVG_AMOUNT_30DAY_WINDOW,TERMINAL_ID_NB_TX_1DAY_WINDOW,TERMINAL_ID_RISK_1DAY_WINDOW,TERMINAL_ID_NB_TX_7DAY_WINDOW,TERMINAL_ID_RISK_7DAY_WINDOW,TERMINAL_ID_NB_TX_30DAY_WINDOW,TERMINAL_ID_RISK_30DAY_WINDOW
0,0,0,2022-01-01 00:00:17,6160,6202,31.83,17,0,0,1,...,1.0,31.83,1.0,31.83,0.0,0.0,0.0,0.0,0.0,0.0
1,1,1,2022-01-01 00:00:31,596,6532,57.16,31,0,0,1,...,1.0,57.16,1.0,57.16,0.0,0.0,0.0,0.0,0.0,0.0
2,2,2,2022-01-01 00:01:05,9339,10651,28.92,65,0,0,1,...,1.0,28.92,1.0,28.92,0.0,0.0,0.0,0.0,0.0,0.0
3,3,3,2022-01-01 00:02:10,4961,8673,81.51,130,0,0,1,...,1.0,81.51,1.0,81.51,0.0,0.0,0.0,0.0,0.0,0.0
4,4,4,2022-01-01 00:02:21,6170,4884,25.17,141,0,0,1,...,1.0,25.17,1.0,25.17,0.0,0.0,0.0,0.0,0.0,0.0


In [52]:
transactions_df.drop(['Unnamed: 0'], axis = 1, inplace= True)
transactions_df.head()

,TRANSACTION_ID,TX_DATETIME,CUSTOMER_ID,TERMINAL_ID,TX_AMOUNT,TX_TIME_SECONDS,TX_TIME_DAYS,TX_FRAUD,TX_DURING_WEEKEND,TX_DURING_NIGHT,...,CUSTOMER_ID_NB_TX_7DAY_WINDOW,CUSTOMER_ID_AVG_AMOUNT_7DAY_WINDOW,CUSTOMER_ID_NB_TX_30DAY_WINDOW,CUSTOMER_ID_AVG_AMOUNT_30DAY_WINDOW,TERMINAL_ID_NB_TX_1DAY_WINDOW,TERMINAL_ID_RISK_1DAY_WINDOW,TERMINAL_ID_NB_TX_7DAY_WINDOW,TERMINAL_ID_RISK_7DAY_WINDOW,TERMINAL_ID_NB_TX_30DAY_WINDOW,TERMINAL_ID_RISK_30DAY_WINDOW
0,0,2022-01-01 00:00:17,6160,6202,31.83,17,0,0,1,1,...,1.0,31.83,1.0,31.83,0.0,0.0,0.0,0.0,0.0,0.0
1,1,2022-01-01 00:00:31,596,6532,57.16,31,0,0,1,1,...,1.0,57.16,1.0,57.16,0.0,0.0,0.0,0.0,0.0,0.0
2,2,2022-01-01 00:01:05,9339,10651,28.92,65,0,0,1,1,...,1.0,28.92,1.0,28.92,0.0,0.0,0.0,0.0,0.0,0.0
3,3,2022-01-01 00:02:10,4961,8673,81.51,130,0,0,1,1,...,1.0,81.51,1.0,81.51,0.0,0.0,0.0,0.0,0.0,0.0
4,4,2022-01-01 00:02:21,6170,4884,25.17,141,0,0,1,1,...,1.0,25.17,1.0,25.17,0.0,0.0,0.0,0.0,0.0,0.0


In [54]:
transactions_df.drop(['TX_TIME_SECONDS', 'TX_TIME_DAYS'], axis = 1, inplace= True)
transactions_df.head()

,TRANSACTION_ID,TX_DATETIME,CUSTOMER_ID,TERMINAL_ID,TX_AMOUNT,TX_FRAUD,TX_DURING_WEEKEND,TX_DURING_NIGHT,CUSTOMER_ID_NB_TX_1DAY_WINDOW,CUSTOMER_ID_AVG_AMOUNT_1DAY_WINDOW,CUSTOMER_ID_NB_TX_7DAY_WINDOW,CUSTOMER_ID_AVG_AMOUNT_7DAY_WINDOW,CUSTOMER_ID_NB_TX_30DAY_WINDOW,CUSTOMER_ID_AVG_AMOUNT_30DAY_WINDOW,TERMINAL_ID_NB_TX_1DAY_WINDOW,TERMINAL_ID_RISK_1DAY_WINDOW,TERMINAL_ID_NB_TX_7DAY_WINDOW,TERMINAL_ID_RISK_7DAY_WINDOW,TERMINAL_ID_NB_TX_30DAY_WINDOW,TERMINAL_ID_RISK_30DAY_WINDOW
0,0,2022-01-01 00:00:17,6160,6202,31.83,0,1,1,1.0,31.83,1.0,31.83,1.0,31.83,0.0,0.0,0.0,0.0,0.0,0.0
1,1,2022-01-01 00:00:31,596,6532,57.16,0,1,1,1.0,57.16,1.0,57.16,1.0,57.16,0.0,0.0,0.0,0.0,0.0,0.0
2,2,2022-01-01 00:01:05,9339,10651,28.92,0,1,1,1.0,28.92,1.0,28.92,1.0,28.92,0.0,0.0,0.0,0.0,0.0,0.0
3,3,2022-01-01 00:02:10,4961,8673,81.51,0,1,1,1.0,81.51,1.0,81.51,1.0,81.51,0.0,0.0,0.0,0.0,0.0,0.0
4,4,2022-01-01 00:02:21,6170,4884,25.17,0,1,1,1.0,25.17,1.0,25.17,1.0,25.17,0.0,0.0,0.0,0.0,0.0,0.0


In [55]:
transactions_df.columns

Index(['TRANSACTION_ID', 'TX_DATETIME', 'CUSTOMER_ID', 'TERMINAL_ID',
       'TX_AMOUNT', 'TX_FRAUD', 'TX_DURING_WEEKEND', 'TX_DURING_NIGHT',
       'CUSTOMER_ID_NB_TX_1DAY_WINDOW', 'CUSTOMER_ID_AVG_AMOUNT_1DAY_WINDOW',
       'CUSTOMER_ID_NB_TX_7DAY_WINDOW', 'CUSTOMER_ID_AVG_AMOUNT_7DAY_WINDOW',
       'CUSTOMER_ID_NB_TX_30DAY_WINDOW', 'CUSTOMER_ID_AVG_AMOUNT_30DAY_WINDOW',
       'TERMINAL_ID_NB_TX_1DAY_WINDOW', 'TERMINAL_ID_RISK_1DAY_WINDOW',
       'TERMINAL_ID_NB_TX_7DAY_WINDOW', 'TERMINAL_ID_RISK_7DAY_WINDOW',
       'TERMINAL_ID_NB_TX_30DAY_WINDOW', 'TERMINAL_ID_RISK_30DAY_WINDOW'],
      dtype='object')

In [56]:
transactions_df.to_csv('transactions_data.csv')